In [7]:
import json
import re
from difflib import SequenceMatcher


def similarity_score(text1, text2):
    """比较两个文本之间的相似度"""
    return SequenceMatcher(None, text1, text2).ratio()


def text_cleaning(text):
    """去除中文文本中的字母、数字和特殊字符"""
    return re.sub('[^\u4e00-\u9fa5]+', '', text)


def remove_emoji(text):
    """去除中文文本中的emoji（通常表示为[哆啦A梦吃惊]）"""
    return re.sub('\[.*\]', '', text)


def duplicating(pre_file, final_file, threshold):
    """去除重复新闻，即将用不同的表述方式描述同一新闻的文本视为同一条文本"""
    lines_seen = set()  #用于存储已经出现过的行
    with open(pre_file, mode='r', encoding='utf-8') as f1, open(final_file, mode='w', encoding='utf-8') as f2:
        for line in f1:
            is_duplicated = False
            # 去除表情、数字、字母、特殊字符
            # line = text_cleaning(remove_emoji(line))
            for line_seen in lines_seen:
                if similarity_score(line, line_seen) >= threshold:
                    is_duplicated = True
                    break
            if not is_duplicated:
                f2.write(line)
                lines_seen.add(line)


def duplicating_nltk():
    pass

In [2]:
# 读取原始文本数据
with open('datasets/rumors_v170613.json', mode='r', encoding='utf-8') as f1, open('datasets/raw_text.txt', mode='w',
                                                                                  encoding='utf-8') as f2:
    for line in f1:
        raw_text = json.loads(line)['rumorText']
        f2.write(raw_text + '\n')

In [3]:
# 数据清洗
with open('datasets/raw_text.txt', mode='r', encoding='utf-8') as f1, open('datasets/pre_text.txt', mode='w',encoding='utf-8') as f2:
    for line in f1:
        line = text_cleaning(remove_emoji(line))
        f2.write(line + '\n')

In [8]:
# 数据去重（对大数据进行去重的话太慢了，看了半集综艺，3万+数据才处理了3千不到
import asyncio

# async def main():
#     await duplicating('datasets/pre_text.txt', 'datasets/final_text.txt', 0.5)

duplicating('datasets/pre_text.txt', 'datasets/final_text.txt', 0.5)

`json.loads()` 和 `json.load()` 函数都用于解析 JSON 数据，但它们有以下不同之处：

1. `json.loads()`：
   - `json.loads()` 是 `json` 模块中的函数，用于将 JSON 字符串解析为 Python 对象。
   - 它接受一个字符串作为输入，并返回一个解析后的 Python 对象，如字典、列表等。
   - 适用于处理 JSON 字符串数据。

2. `json.load()`：
   - `json.load()` 是 `json` 模块中的函数，用于从文件中读取 JSON 数据并解析为 Python 对象。
   - 它接受一个文件对象作为输入，并返回一个解析后的 Python 对象，如字典、列表等。
   - 适用于处理存储在文件中的 JSON 数据。

总结：
- `json.loads()` 用于解析 JSON 字符串，接受字符串作为输入。
- `json.load()` 用于从文件中读取 JSON 数据并解析，接受文件对象作为输入。

需要注意的是，`json.load()` 函数需要通过文件操作打开和关闭文件，而 `json.loads()` 函数直接处理字符串数据，不涉及文件操作。

In [ ]:
# 使用python标准库进行数据去重的简单范例
def similarity_score(text1, text2):
    return SequenceMatcher(None, text1, text2).ratio()


def data_cleaning(data, threshold):
    cleaned_data = []
    processed_texts = []

    for text in data:
        text = text.strip().lower()  # 预处理文本，去除空格，转换为小写
        is_duplicate = False
        for processed_text in processed_texts:
            score = similarity_score(text, processed_text)
            if score > threshold:
                is_duplicate = True
                break
        if not is_duplicate:
            cleaned_data.append(text)
            processed_texts.append(text)

    return cleaned_data


# 示例数据
data = [
    "京海抢孩子",
    "京海抢孩子了",
    "京海光天化日抢孩子",
    "这是一条不相似的文本"
]
# 设置相似度阈值
threshold = 0.6
# 进行数据清洗
cleaned_data = data_cleaning(data, threshold)
# 输出清洗后的数据
for text in cleaned_data:
    print(text)

在SequenceMatcher类的构造函数中，第一个参数用于传递一个比较函数，指定如何比较两个序列的元素。

该参数的默认值为None，当设置为None时，SequenceMatcher会使用默认的比较函数，即逐个比较序列中的元素。

如果需要自定义比较函数，可以将自定义的函数作为第一个参数传递给SequenceMatcher的构造函数。该自定义函数需要接受两个参数，并返回用于比较的结果。自定义函数的形式可以是一个普通函数或者是一个lambda函数。

In [2]:
import nltk
from nltk.metrics.distance import edit_distance # 编辑距离
from nltk.tokenize import word_tokenize     #分词（词元化）
from nltk.corpus import stopwords       #停用词
from collections import defaultdict     #导入子类defaultdict

"""defaultdict是一个字典（dictionary）的子类，它重写了字典的一些方法，并添加了一个特殊的功能：当访问字典中不存在的键时，它可以返回一个默认值。这个默认值在创建defaultdict对象时通过一个参数指定"""

# nltk.download('punkt')    # 用于分句的标记器模块，将文本分割成句子
# nltk.download('stopwords')    #包含多种语言的停用词列表，用于在文本处理中过滤常见的无意义词汇


def build_inverted_index(data):
    """使用倒排索引构建了一个索引数据结构"""
    inverted_index = defaultdict(list)  #定义字典中的值为list格式
    for i, text in enumerate(data):
        tokens = set(word_tokenize(text))   #对text进行分词，然后存入集合
        for token in tokens:
            inverted_index[token].append(i) #对同一行的每一个分词都插入同样的索引值
    return inverted_index


def approximate_match(query, data, inverted_index, threshold):
    candidates = set()  # 创建一个集合
    query_tokens = set(word_tokenize(query))    #待查询的词元集合
    for token in query_tokens:
        candidates.update(inverted_index[token])#将待查询的词元对应的索引插入集合
    results = []
    for candidate in candidates:
        distance = edit_distance(query, data[candidate])
        similarity = 1 - distance / max(len(query), len(data[candidate]))
        if similarity >= threshold:
            results.append((data[candidate], similarity))
    return results


input_file = "datasets/pre_text.txt"
output_file = "datasets/final_text.txt"
similarity_threshold = 0.5

# 读取文本数据
with open(input_file, "r", encoding='utf-8') as f:
    data = [line.strip() for line in f]

# 构建倒排索引
inverted_index = build_inverted_index(data)

# 文本去重
seen = set()
deduplicated_data = []
for text in data:
    if text not in seen:
        deduplicated_data.append(text)
        seen.add(text)

# 使用近似字符串匹配进行去重
deduplicated_data = []
seen = set()
stop_words = set(stopwords.words("chinese"))

for text in data:
    tokens = [token for token in word_tokenize(text) if token not in stop_words]
    processed_text = " ".join(tokens)
    if processed_text not in seen:
        deduplicated_data.append(text)
        seen.add(processed_text)

# 写入输出文件
with open(output_file, "w", encoding='utf-8') as f:
    for text in deduplicated_data:
        f.write(text + "\n")

In [8]:
"""使用倒排索引构建了一个索引数据结构"""
with open('datasets/pre_text.txt',mode='r',encoding='utf-8') as f:
    data = [line.strip() for line in f][:2]
    # print(data[:20])
    inverted_index = defaultdict(list)  #定义字典中的值为list格式
    for i, text in enumerate(data):
        tokens = set(word_tokenize(text))   #对text进行分词，然后存入集合
        print(tokens)
        for token in tokens:
            inverted_index[token].append(i) #插入索引？
    print(inverted_index)

{'寻人启事', '求转', '今天', '扩散', '实验小学', '帮忙', '三岁多', '小女孩', '急找', '孩子', '在', '上午', '一个'}
{'有', '感谢', '姐妹', '联系人', '四十多岁', '的', '男人', '懒得', '兄弟', '看一眼', '拐走了', '知情者', '信息', '她', '监控', '从', '抱走了', '急疯了', '爸爸', '大人', '一下', '一个', '张静杰', '能', '小女孩', '锦绣花园', '附近', '被人', '万分', '现', '请告之', '被', '如果', '都', '说出', '是', '手机号码', '上', '看到', '留意', '小区', '看'}
defaultdict(<class 'list'>, {'寻人启事': [0], '求转': [0], '今天': [0], '扩散': [0], '实验小学': [0], '帮忙': [0], '三岁多': [0], '小女孩': [0, 1], '急找': [0], '孩子': [0], '在': [0], '上午': [0], '一个': [0, 1], '有': [1], '感谢': [1], '姐妹': [1], '联系人': [1], '四十多岁': [1], '的': [1], '男人': [1], '懒得': [1], '兄弟': [1], '看一眼': [1], '拐走了': [1], '知情者': [1], '信息': [1], '她': [1], '监控': [1], '从': [1], '抱走了': [1], '急疯了': [1], '爸爸': [1], '大人': [1], '一下': [1], '张静杰': [1], '能': [1], '锦绣花园': [1], '附近': [1], '被人': [1], '万分': [1], '现': [1], '请告之': [1], '被': [1], '如果': [1], '都': [1], '说出': [1], '是': [1], '手机号码': [1], '上': [1], '看到': [1], '留意': [1], '小区': [1], '看': [1]})


倒排序索引（Inverted Index）是一种常用的文本索引数据结构，主要用于快速地搜索和查找文档中的关键字。

相比于传统的正向索引（Forward Index），倒排序索引将文档中的每个单词作为一个索引项，然后记录下每个单词在哪些文档中出现过。这样就可以通过搜索关键字来快速定位包含该关键字的文档，从而实现高效的文本检索。

具体来说，倒排序索引由两部分组成：词项表和倒排列表。词项表中存储了所有单词及其对应的倒排列表的指针，倒排列表则记录了每个单词在哪些文档中出现过以及出现的位置信息。

举个例子，如果有三篇文档：

- 文档1：The quick brown fox jumps over the lazy dog.
- 文档2：A quick red fox jumps over the lazy dog.
- 文档3：A brown fox jumps over the lazy dog.

那么构建倒排序索引的过程如下：

1. 分离文档中的单词，得到所有的词项：

```
quick, brown, fox, jumps, over, the, lazy, dog, a, red
```

2. 对于每个词项，记录它在哪些文档中出现过，并且出现的位置信息：

```
quick: 1,2 (1:2, 2:1)
brown: 1,3 (1:3, 3:1)
fox: 1,2,3 (1:4, 2:3, 3:2)
jumps: 1,2,3 (1:5, 2:4, 3:3)
over: 1,2,3 (1:6, 2:5, 3:4)
the: 1,2 (1:1, 2:5)
lazy: 1,2,3 (1:7, 2:6, 3:5)
dog: 1,2,3 (1:8, 2:7, 3:6)
a: 2,3 (2:1, 3:1)
red: 2 (2:2)
```

最终得到的倒排序索引就是这样的，我们可以通过关键字查找对应的倒排列表，然后定位到相应的文档和位置。例如，如果要查找包含单词“brown”的文档，只需要在倒排列表中查找“brown”，然后找到它对应的文档编号即可。

edit_distance也被称为Levenshtein距离，是一种衡量两个字符串之间的差异程度的方法。它定义为将一个字符串变换成另一个字符串所需的最少操作数，其中每个操作可以是插入一个字符、删除一个字符或替换一个字符。

这个算法使用动态规划来计算两个字符串之间的编辑距离。假设我们有两个字符串s和t，长度分别为m和n。我们使用一个(m+1)×(n+1)的矩阵D来记录编辑距离。矩阵中的每个元素D(i,j)表示将s的前i个字符转换为t的前j个字符需要的最少编辑距离。

具体地，我们可以通过以下方式递归地计算D(i,j)：

1. 如果i=0，那么D(i,j)=j，因为将空字符串转换为t中的前j个字符需要插入j个字符。

2. 如果j=0，那么D(i,j)=i，因为将s中的前i个字符转换为空字符串需要删除i个字符。

3. 如果s[i]=t[j]，那么D(i,j)=D(i-1,j-1)，因为这两个字符匹配，不需要进行任何操作。

4. 如果s[i]!=t[j]，那么我们可以尝试三种操作：

   a. 插入操作：将t[j]插入s中的第i+1个位置，D(i,j)=D(i,j-1)+1。

   b. 删除操作：删除s[i]，D(i,j)=D(i-1,j)+1。

   c. 替换操作：将s[i]替换为t[j]，D(i,j)=D(i-1,j-1)+1。

   我们选择这三个操作中最小的一个，并将其作为D(i,j)的值。

最终，D(m,n)就是字符串s和t之间的编辑距离。

例如，对于字符串s="kitten" 和 t="sitting"，我们可以构建以下矩阵：

| | |s|i|t|t|i|n|g|
|-|-|-|-|-|-|-|-|-|
| |0|1|2|3|4|5|6|7|
|k|1|1|2|3|4|5|6|7|
|i|2|2|1|2|3|4|5|6|
|t|3|3|2|1|2|3|4|5|
|t|4|4|3|2|1|2|3|4|
|e|5|5|4|3|2|2|3|4|
|n|6|6|5|4|3|3|2|3|

因此，编辑距离为3。